In [11]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from pymongo import MongoClient
import pandas as pd
import numpy as np
import json
import difflib
import tqdm
import time
from get_categories import get_categories
from get_transactions import get_transaction_data

transactions_df = get_transaction_data()
project_smc_df = get_categories(True)

transaction_project_df = pd.merge(transactions_df,
                                project_smc_df,
                                left_on='to_address', 
                                right_on='address', 
                                how='inner')

address_categories = transaction_project_df.groupby(["mapped_categories", "from_address"]).count().reset_index()[['mapped_categories', 'from_address', "to_address", '_id_x']]
address_categories.columns = ["categories", "user_address", "projectAddress", "count"]

for cate in address_categories.categories.unique():
    address_categories[f'{cate}'] = len(address_categories[address_categories.categories == cate])
    
address_categories.to_csv("address_categories.csv", index=False)

/home/ubuntu/system/Dev/GroupProjectbyCentroid/get_transactions.py:6: DtypeWarning: Columns (17,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions = pd.read_csv("transactions.csv")


abc


/home/ubuntu/system/Dev/GroupProjectbyCentroid/get_categories.py:9: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  smart_contracts_df = pd.read_csv("project_smc.csv")


def


In [2]:
address_categories = transaction_project_df.groupby(["mapped_categories", "from_address"]).count().reset_index()[['mapped_categories', 'from_address', '_id_x']]
address_categories.columns = ["categories", "user_address", "count"]

In [10]:
for cate in address_categories.categories.unique():
    address_categories[f'{cate}'] = len(address_categories[address_categories.categories == cate])
    
address_categories.to_csv("address_categories.csv", index=False)

NameError: name 'address_categories' is not defined

In [4]:
address_categories.head()

,categories,userAddress,count,Art_count,Derivatives_count,Dexes_count,Gaming_count,Infrastructure_count,Lending_count,NFT_count,Services_count,Stablecoins_count,Virtual Worlds_count
0,Art,0x0012195f694f8125438c03c1692917e6de5fda76,1,2187,1535,348,115,23099,513,3272,1354,2644,809
1,Art,0x0021fedbd4d45d9762a2aadc4e4837ac949e8bb0,1,2187,1535,348,115,23099,513,3272,1354,2644,809
2,Art,0x00256d6f9757a193778c21eda9f8df4f74f47e7d,1,2187,1535,348,115,23099,513,3272,1354,2644,809
3,Art,0x002e1798bff1ea5bcd703133eb61706070080c19,1,2187,1535,348,115,23099,513,3272,1354,2644,809
4,Art,0x004b883f8adf1d0fabafc129e9ff9a43fea525fc,1,2187,1535,348,115,23099,513,3272,1354,2644,809


In [5]:
spark = SparkSession.builder.appName("test").getOrCreate()
df = spark.read.options(inferSchema='True',delimiter=',', header='True').csv("address_categories.csv")

feature_cols = df.columns
feature_cols.remove("categories")
feature_cols.remove("userAddress")

va = VectorAssembler(inputCols= feature_cols,
                    outputCol = "features")
kmeans = KMeans()

model = kmeans.fit(va.transform(df))
# pipeline = Pipeline(stages=[va, kmeans])

# pipeline_model = pipeline.fit(df)

In [6]:
# res = pipeline_model.transform(df).select("address", "prediction").toPandas()
centroids = model.clusterCenters()

distance_columns = [
    F.sqrt(sum((F.col(feature_cols[i]) - centroids[j][i]) ** 2 for i in range(len(feature_cols))))
    for j in range(len(centroids))
]

# Add a column 'distance_to_centroid' to the DataFrame
df_with_distance = model.transform(va.transform(df)).withColumn(
    "distance_to_centroid",
    distance_columns[model.transform(va.transform(df)).select("prediction").collect()[0]["prediction"]]
)


In [7]:
res_df = df_with_distance.select("userAddress", "prediction", "distance_to_centroid").toPandas()

In [ ]:
def infer(df):
    global pipeline_model, project_smc_df
    res: pd.DataFrame = pipeline_model.transform(df).select("address", "prediction").toPandas()
    
    

In [17]:
merged_df = pd.merge(res_df,
                     transaction_project_df[["from_address", "to_address"]],
                     left_on='userAddress',
                     right_on='from_address',
                     how='inner')

In [22]:
merged_df.columns

Index(['userAddress', 'prediction', 'distance_to_centroid', 'from_address',
       'to_address'],
      dtype='object')

In [21]:
merged_df.groupby("prediction")[['to_address', 'distance_to_centroid']].agg(list).reset_index()

,prediction,to_address,distance_to_centroid
0,0,"[0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce, 0...","[0.49535681418890665, 0.49535681418890665, 0.4..."
1,1,"[0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9, 0...","[173.5046431858111, 173.5046431858111, 173.504..."


In [16]:
transaction_project_df.columns

Index(['Unnamed: 0_x', '_id_x', 'type', 'hash', 'nonce', 'transaction_index',
       'from_address', 'to_address', 'value', 'gas', 'gas_price', 'input',
       'block_timestamp', 'block_number', 'block_hash',
       'receipt_cumulative_gas_used', 'receipt_gas_used',
       'receipt_contract_address', 'receipt_root', 'receipt_status',
       'item_timestamp', 'decoded_input_event', 'enriched_data', 'event_types',
       'related_addresses_event', 'decoded_input', 'related_addresses',
       'transaction_type', 'Unnamed: 0_y', '_id_y', 'address', 'name',
       'symbol', 'categories', 'chainId', 'mapped_categories'],
      dtype='object')

In [12]:
import pandas as pd

df = pd.read_csv("save.csv")
address = pd.read_csv("address_categories.csv")


In [13]:
address.head()

,categories,user_address,Art,Cexes,Derivatives,Gaming,Infrastructure,Lending,NFT,Services,Stablecoins,Virtual Worlds,Indexes,Membership,PFPs,Dexes
0,Art,0x0012195f694f8125438c03c1692917e6de5fda76,2187,348,1535,115,23099,513,3272,1354,2644,809,0,0,0,0
1,Art,0x0021fedbd4d45d9762a2aadc4e4837ac949e8bb0,2187,348,1535,115,23099,513,3272,1354,2644,809,0,0,0,0
2,Art,0x00256d6f9757a193778c21eda9f8df4f74f47e7d,2187,348,1535,115,23099,513,3272,1354,2644,809,0,0,0,0
3,Art,0x002e1798bff1ea5bcd703133eb61706070080c19,2187,348,1535,115,23099,513,3272,1354,2644,809,0,0,0,0
4,Art,0x004b883f8adf1d0fabafc129e9ff9a43fea525fc,2187,348,1535,115,23099,513,3272,1354,2644,809,0,0,0,0


In [5]:
df.head()

,user_address,user_name,twitter_username,twitter_name,Art,Cexes,Derivatives,Dexes,Gaming,Indexes,Infrastructure,Lending,Memberships,PFPs,Services,Stablecoins,Virtual Worlds,Yield
0,0,IndahWulandari,indahwullannd,Indah Wulandari🌱Freetag,1,0,0,1,0,0,0,0,0,1,0,0,0,0
1,0x00000CDFC978ad918f240581aDb878E17299809F,MikaPrjkt,0xOyen,💚🌙 $GEKKO ❤️ Memecoin #Upbot $UP MagicSquare,3,0,0,1,1,0,0,0,0,3,0,0,0,1
2,0x0018e67F5Ce738E0177ee667dd20516D7d125cb6,jeromenichole,hanaokalagi,hanaokalagi,1,0,0,1,0,0,0,0,1,0,0,0,0,0
3,0x001a1A6126aDFD06637a75bC08eD0bc4d9F3de87,disund.bnb,DickSundgren,disund 💬beoble : ⚔️ MagicSquare,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0x001f90fa9B45B3251A7AE692B7A02376AD931a1D,0,Solfrens20,Solfrens (20),1,0,0,0,0,0,1,0,0,1,0,0,0,0


In [7]:
address[address.userAddress.isin(df.user_address)]

,categories,userAddress,count,Art_count,Derivatives_count,Dexes_count,Gaming_count,Infrastructure_count,Lending_count,NFT_count,Services_count,Stablecoins_count,Virtual Worlds_count


In [6]:
pd.merge(df, address, left_on='user_address',  right_on='userAddress', how='inner')

,user_address,user_name,twitter_username,twitter_name,Art,Cexes,Derivatives,Dexes,Gaming,Indexes,...,Art_count,Derivatives_count,Dexes_count,Gaming_count,Infrastructure_count,Lending_count,NFT_count,Services_count,Stablecoins_count,Virtual Worlds_count


In [7]:
cate = ["Art",
               "Cexes",
               "Derivatives",
               "Dexes",
               "Gaming", 
               "Indexes",
               "Infrastructure",
               "Lending",
               "Memberships",
               "PFPs",
               "Services",
               "Stablecoins",
               "Virtual Worlds",
               "Yield"]
for column in cate:
    indices_with_feature = df[column].idxmax()
    print(f"Indices where feature '{column}' is equal to 1:", indices_with_feature)


Indices where feature 'Art' is equal to 1: 1781
Indices where feature 'Cexes' is equal to 1: 0
Indices where feature 'Derivatives' is equal to 1: 0
Indices where feature 'Dexes' is equal to 1: 1781
Indices where feature 'Gaming' is equal to 1: 1
Indices where feature 'Indexes' is equal to 1: 0
Indices where feature 'Infrastructure' is equal to 1: 4
Indices where feature 'Lending' is equal to 1: 0
Indices where feature 'Memberships' is equal to 1: 1094
Indices where feature 'PFPs' is equal to 1: 1781
Indices where feature 'Services' is equal to 1: 6
Indices where feature 'Stablecoins' is equal to 1: 0
Indices where feature 'Virtual Worlds' is equal to 1: 0
Indices where feature 'Yield' is equal to 1: 966


In [ ]:
df.column

In [8]:
df['active_feature'] = df[cate].apply(lambda row: row.idxmax(), axis=1)

In [9]:
df.head()

,user_address,user_name,twitter_username,twitter_name,Art,Cexes,Derivatives,Dexes,Gaming,Indexes,Infrastructure,Lending,Memberships,PFPs,Services,Stablecoins,Virtual Worlds,Yield,active_feature
0,0,IndahWulandari,indahwullannd,Indah Wulandari🌱Freetag,1,0,0,1,0,0,0,0,0,1,0,0,0,0,Art
1,0x00000CDFC978ad918f240581aDb878E17299809F,MikaPrjkt,0xOyen,💚🌙 $GEKKO ❤️ Memecoin #Upbot $UP MagicSquare,3,0,0,1,1,0,0,0,0,3,0,0,0,1,Art
2,0x0018e67F5Ce738E0177ee667dd20516D7d125cb6,jeromenichole,hanaokalagi,hanaokalagi,1,0,0,1,0,0,0,0,1,0,0,0,0,0,Art
3,0x001a1A6126aDFD06637a75bC08eD0bc4d9F3de87,disund.bnb,DickSundgren,disund 💬beoble : ⚔️ MagicSquare,1,0,0,0,0,0,0,0,0,1,0,0,0,0,Art
4,0x001f90fa9B45B3251A7AE692B7A02376AD931a1D,0,Solfrens20,Solfrens (20),1,0,0,0,0,0,1,0,0,1,0,0,0,0,Art


In [14]:
import pandas as pd
import numpy as np
import ast

df = pd.read_csv("res.csv")

In [12]:
df['distance_to_centroid'] = df['distance_to_centroid'].apply(lambda x: [value + np.random.rand() for value in x])


In [6]:
df['distance_to_centroid'][0]

'['

In [13]:
df.to_csv("res.csv", index=False)

In [4]:
import pandas as pd

# Sample DataFrame with lists in columns A and B
data = {'A': [[10, 5, 20], [5, 15, 10], [20, 10, 15], [15, 25, 20], [25, 20, 15]],
        'B': [['John', 'Alice', 'Bob'], ['Eve', 'Charlie', 'David'], ['Alice', 'John', 'Charlie'],
              ['David', 'Eve', 'Bob'], ['Charlie', 'Bob', 'Alice']]}

df = pd.DataFrame(data)

# Define a function to sort both columns A and B based on the values in column A
def sort_columns(row):
    sorted_values = sorted(zip(row['A'], row['B']), reverse=True)[:1]
    row['A'], row['B'] = zip(*sorted_values)
    return row

# Apply the function to each row
df[['A', "B"]] = df[['A', "B"]].apply(sort_columns, axis=1)
# Display the modified DataFrame
print("DataFrame with sorted columns A and B:")
print(df[['A', 'B']])


DataFrame with sorted columns A and B:
       A           B
0  (20,)      (Bob,)
1  (15,)  (Charlie,)
2  (20,)    (Alice,)
3  (25,)      (Eve,)
4  (25,)  (Charlie,)
